# Introduction and Background

In this notebook, we introduce t-SNE, a probability-distribution preserving dimensionality reduction technique, as well as UMAP, a technique which yields a similar result in a short time.

**Note**: To display values of variables in the markdown cells, you should [enable the jupyter contrib nbextension](https://stackoverflow.com/questions/52812231/print-variable-in-jupyter-notebook-markdown-cell-python).

In [ ]:
#!/usr/bin/env python3

import sys

# Maths things
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Local Utilities for Notebook
sys.path.append("../")
from utilities.general import load_variables, sorted_eig, get_stats
from utilities.plotting import (
    plot_projection,
    plot_regression,
    check_mirrors,
    get_cmaps,
    table_from_dict,
)

from sklearn.metrics.pairwise import linear_kernel, rbf_kernel
from functools import partial

from skcosmo.preprocessing import KernelNormalizer
from sklearn.decomposition import PCA, KernelPCA
from openTSNE import TSNE
import umap
from functools import partial

from skcosmo.preprocessing import KernelNormalizer

cmaps = get_cmaps()
plt.style.use("../utilities/kernel_pcovr.mplstyle")
dbl_fig = (2 * plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])

First, we must load the data. For a step-by-step explanation of this, please see [Importing Data](X_ImportingData.ipynb). As mentioned in the [foreword notebook](0_Foreword.ipynb), pre-computed features should be downloaded from [here](https://www.dropbox.com/s/itokckbbkvxaqsk/precomputed.npz?dl=0) and the file `precomputed.npz` should be copied to the `datasets/` folder.

In [ ]:
var_dict = load_variables()
locals().update(var_dict)

# t-SNE

The aim of [t-distribution stochastic neighbor embedding](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) (t-SNE) [[1]](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf), is to reproduce the probability distribution of the original feature space. Preserving the probability distribution of feature space can be particularly important in simulations when modeling or observing thermodynamic quantities, as one might use a low-dimensional map to understand the local landscape.
We start from the conditional probability distribution that points $j$ is a neighbor to points $i$ and assuming an underlying Gaussian distribution:

$$p(j | i) =\frac{\exp[d(x_i, x_j)/2\sigma^2]}{sum_{k\neq i}{exp[d(x_i, x_j)/2\sigma^2]}}$$

Where $\sigma$ is our Gaussian kernel width, and $d(x_i, x_j)$ is a distance metric. This distance metric is often, but not necessarily, the Euclidean distance $d(x_i, x_j) = || x_i - x_j||^2$.  The probability distribution $p_{ij}$ is then given by the normalized sum of the conditional probabilities using $p(j|i)$ and $p(i|j)$.

We then map $p_{ij}$ onto a lower-dimension distribution $q_{ij}$ by minimizing the [Kullback-Leibler](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) divergence between the two distributions.

$$KL(p, q) = sum_{ij} \left[p_{ij}log\left(\frac{p_{ij}}{q_{ij}}\right)\right]$$

There are many packages for computing t-SNE (including a [package available](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) in scikit-learn), here we use [openTSNE](https://opentsne.readthedocs.io/en/latest/) for its computational efficiency and its focus on algorithm convergence.

For readers wishing to know more about the ins and outs of t-SNE, there is an excellent article written by [Wattenberg, et al.](https://distill.pub/2016/misread-tsne/) that steps through the importance of each hyperparameter very nicely.

### t-SNE vs. PCA

In [ ]:
tsne = TSNE(
    n_components=2, # dimensionality of q
    perplexity=10, # how to balance between local neighborhoods and global shape
    metric="euclidean", # distance metric
    n_jobs=2, # parallelization
    random_state=42, # random seed
)
T_tsne_train=tsne.fit(X_train)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4), gridspec_kw=dict(width_ratios=(1, 1.5)))
plot_projection(
    numbers[i_train], T_tsne_train, title="t-SNE of Training Data", 
    x_label="t-SNE 1",
    y_label="t-SNE 2",
    cbar=True,
    cbar_title="Atomic\nNumber",
    fig=fig,
    ax=ax[1],
    s=10,
    **cmaps
)
ax[1].set_aspect('auto')

plot_projection(
    numbers[i_train], PCA(n_components=2).fit_transform(X_train), title="PCA of Training Data", 
    cbar=False,
    fig=fig,
    ax=ax[0],
    s=10,
    **cmaps
)
ax[0].set_xticks([])
ax[1].set_xticks([])
ax[0].set_yticks([])
ax[1].set_yticks([])
fig.subplots_adjust(wspace=0.1, hspace=0.1)

Here we can see how the t-SNE projection differs from the PCA. We still get the separation of data points due to chemical identity; however, the clusters of hydrogens and carbons are much more stretched out, likely due to the greater variability of atomic environments for these two atomic species. 

It's not necessary to use the euclidean distance in t-SNE, and we can similarly modify it to incorporate non-linearity like we did PCA!

## UMAP

Uniform Manifold Approximation and Projection (UMAP) is a dimensionality reduction technique, introduced first by [McInnes and Healy in 2018](https://joss.theoj.org/papers/10.21105/joss.00861), aimed to recreate the underlying manifold of samples in a lower-dimensional space. UMAP will obtain a visually similar result to t-SNE, but uses tricks of topological data analyses to drastically reduce the computational overhead. Given the complexity of these mathematics, the implementation of UMAP is beyond the scope of this text, but we urge interested readers to dive into the derivation found in [McInnes and Healy in 2018](https://joss.theoj.org/papers/10.21105/joss.00861) or in the online documentation to the [UMAP software package](https://umap-learn.readthedocs.io). 

In UMAP, we first assume that all data points lie uniformly distributed on a high-dimensional manifold. On this manifold, we extend a radius from each point. Any two points with overlapping radii are assumed to be neighbors, and each point is enforced to have a minimum number of neighbors. In UMAP, we assume these radii to be “fuzzy”, such that the likelihood of being neighbors decreases with distance from each point. This procedure results in a graph of points; the resulting low-dimensional embedding is that which best preserves this graph structure.

In [ ]:
n_neigh = np.logspace(0, 2, 5, dtype=int)
n_neigh[0] = 2 # cannot have only one neighbor

fig, axes = plt.subplots(1, len(n_neigh),figsize=(2*len(n_neigh),2))

for n, ax in zip(n_neigh, axes):
    u = umap.UMAP(n_neighbors=n).fit_transform(X_train)
    plot_projection(
                    numbers[i_train], u, 
                    title=f"n={n}", 
                    x_label="UMAP 1",
                    y_label="UMAP 2",
                    cbar=False,
                    fig=fig,
                    ax=ax,
                    s=10,
                    **cmaps
                    )
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
fig.show()

As the number of neighbors increases, the clusters separate further. This is because the clusters are each becoming self-isolated as the neighborhoods are more clearly identified.